In [1]:
import pandas as pd
import numpy as np
from astropy.time import Time
from math import radians, cos, sin, asin, sqrt

In [42]:
csvfile = "../../Data/Location/ISS_FERMI_Light-1_Locations.csv"

data_pd = pd.read_csv(csvfile)
headers = list(data_pd.keys())
data    = [i for i in data_pd.to_numpy().T]
data    = dict(zip(headers,data))

In [109]:
# headers
# data

In [24]:
def haversine(lon1, lat1, lon2, lat2):  #Calculate the great circle distance between two points on the earth 
                                        #(specified in decimal degrees)
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2]) # convert decimal degrees to radians 
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles.
    return c * r

In [52]:
distance_ISS_Light1 = [haversine(data['Lon(deg)Light-1'][i],data['Lat(deg)Light-1'][i],data['Lon(deg)ISS'][i],data['Lat(deg)ISS'][i]) for i in range(len(data['Time(ModJDate)']))]
distance_FERMI_Light1 = [haversine(data['Lon(deg)Light-1'][i],data['Lat(deg)Light-1'][i],data['Lon(deg)FERMI'][i],data['Lat(deg)FERMI'][i]) for i in range(len(data['Time(ModJDate)']))]

In [116]:
THRESHOLD_km   = 1000
ISS_idx_cut     = np.where([i < THRESHOLD_km for i in distance_ISS_Light1])[0]
ISS_time_cut    = data['Time(ModJDate)'][ISS_idx_cut]

FERMI_idx_cut     = np.where([i < THRESHOLD_km for i in distance_FERMI_Light1])[0]
FERMI_time_cut    = data['Time(ModJDate)'][FERMI_idx_cut]

In [115]:
ISS_time = [Time(i,format="mjd") for i in ISS_time_cut]
ISS_time = [(i.mjd,i.isot) for i in ISS_time]

FERMI_time = [Time(i,format="mjd") for i in FERMI_time_cut]
FERMI_time = [(i.mjd,i.isot) for i in FERMI_time]

In [117]:
# .00300s is 5 minutes.. so 1000s for safety
TIME_THRESHOLD = 0.01

ISS_time_list = [('ISS Start here:', ISS_time[0][1])]
ISS_val = 0 # To decide if start or end
for i in range(len(ISS_time)-1):
    if ((ISS_time[i+1][0] - ISS_time[i][0]) >= TIME_THRESHOLD):

        if ((ISS_val % 2) == 0): # If val is even
            k = ("End here:", ISS_time[i][1])
            
        else:
            k = ("Start here:", ISS_time[i+1][1])
        
        ISS_time_list.append(k)
        ISS_val = ISS_val + 1

FERMI_time_list = [('FERMI Start here:', FERMI_time[0][1])]
FERMI_val = 0 # To decide if start or end
for i in range(len(FERMI_time)-1):
    if ((FERMI_time[i+1][0] - FERMI_time[i][0]) >= TIME_THRESHOLD):

        if ((val % 2) == 0): # If val is even
            k = ("End here:", FERMI_time[i][1])
            
        else:
            k = ("Start here:", FERMI_time[i+1][1])
        
        FERMI_time_list.append(k)
        val = val + 1


In [119]:
# ISS_time_list
# FERMI_time_list